# Task01 决策树（上）

## 1 知识梳理

### 1.1 信息论基础
- 决策树思想：通过判定条件对节点内的样本进行划分，通过节点内部纯度规则，进行分类
- 节点纯度：节点样本标签的不确定性
- 不确定性的度量：$$H(p_1,\cdots,p_n) = -C \sum_{i=1}^n p_i \log p_i$$
- 三个信息熵条件：
  1. $H$是关于$p_i$的连续函数
  2. 若$p_1=p_2=\cdots=p_n$，则$H$关于$n$单调递增
  3. 若将某一个$p_i$拆分为$p_{i1}$和$p_{i2}$，即$p_{i1}+p_{i2}=p_i$，则$$
  H(p_1,\cdots,p_{i-1},p_{i+1},\cdots,p_n,p_{i1}, p_{i2})=H(p_1, \cdots, p_n) + p_i H(\frac{p_{i1}}{p_i}, \frac{p_{i2}}{p_i})
  $$

- 条件熵：$H(Y|X) = E_{X}[E_{Y|X}[-\log_2 p(Y|X)]]$
- 信息增益：$$
\begin{aligned}
G(Y,X)
&= H(Y) - H(Y|X) \\
&= -\sum_{i=1}^K \tilde{p}(y_k) \log_2 \tilde{p}(y_k) + \sum_{m=1}^M \tilde{p}(x_m) \sum_{k=1}^K \tilde{p}(y_k|x_m) \log_2 \tilde{p}(y_k|x_m)
\end{aligned}
$$

### 1.2 分类树的节点分裂

- 类别特征：每个节点选择最大信息增益$G_N^{max}(Y,X)$对应的特征进行分裂，直到所有节点的相对最大信息增益$\displaystyle \frac{D_N}{D_{all}}G_N^{max}(Y,X)$小于$\epsilon$
- 数值特征：
  1. 最佳分割法：遍历所有的$x$，按照特征$x$中元素与$s$（$s \sim U[x_{min}, x_{max}]$）比较，分为两个集合，计算所有$s$对应信息增益最大值，将该值作为特征分裂信息增益
  2. 随机分割法：通过将节点样本按照特征$x$中元素与$s$（$s \sim U[x_{min}, x_{max}]$）比较，分为两个集合，转变为类别特征处理

- C4.5算法：使用信息增益比代替信息增益，信息增益比：$\displaystyle G^R(Y,X)=\frac{G(Y,X)}{H(X)}$

- sklearn提供两种生长模式：深度优先生长、最佳增益生长

### 1.3 CART树

- 概念：一棵二叉树，既能处理分类问题，又能处理回归问题
- 回归问题处理
  1. 节点纯度定义：使用节点间元素大小表示，接近程度代表纯度
  2. 信息增益：$$
  G^{MSE}(Y,X)=\frac{1}{N}\sum_{i=1}^{N}(y^{(D)}_i-\bar{y}^{(D)})^2-\frac{N_L}{N}\frac{1}{N_L}\sum_{i=1}^{N_L}(y^{(L)}_i-\bar{y}^{(L)})^2-\frac{N_R}{N}\frac{1}{N_R}\sum_{i=1}^{N_R}(y^{(R)}_i-\bar{y}^{(R)})^2 \\
  G^{MAE}(Y,X)=\frac{1}{N}\sum_{i=1}^{N}\vert y^{(D)}_i-\tilde{y}^{(D)}\vert-\frac{N_L}{N}\frac{1}{N_L}\sum_{i=1}^{N_L}\vert y^{(L)}_i-\tilde{y}^{(L)}\vert-\frac{N_R}{N}\sum_{i=1}^{N_R}\frac{1}{N_R}\vert y^{(R)}_i-\tilde{y}^{(R)}\vert
  $$
- 分类问题处理：
  1. 通过将熵中的log在$p=1$处一阶泰勒展开，计算近似的基尼系数
  2. 基于基尼系数的信息增益：$$G(Y,X) = \text{Gini}(Y) - \text{Gini}(Y|X)$$

### 1.4 决策树的剪枝

- 预剪枝：树在判断节点是否分裂的时候就预先通过一些规则来阻止其分裂。
- 后剪枝：在树的节点已经全部生长完成后，同故宫一些规则来摘除一些子树。

## 2 练习

### 2.1 练习1
定义$X,Y$的联合熵为$H(Y,X)$为$\mathbb{E}_{(Y,X)\sim p(y,x)}[-\log_2p(Y,X)]$
1. 请证明如下关系：
$$
G(Y,X) = H(X) - H(X|Y) \\
G(Y,X) = H(X) + H(Y) - H(Y,X) \\
G(Y,X) = H(Y,X) -H(X|Y) -H(Y|X)
$$
2. 下图被分为了$A$、$B$和$C$三个区域。若$AB$区域代表$X$的不确定性，$BC$区域代表$Y$的不确定性，那么$H(X)、H(Y)、H(X|Y)、H(Y|X)、H(Y,X)和G(Y,X)$分别指代的是哪片区域？

![练习1](images/task01/tree_pic3.png)

**解答：**

**第一问：**
$$
H(X) = E_Y[-\log_2 p(Y)] = -\sum_{k=1}^K p(y_k) \log_2 p(y_k) \\
H(Y|X) = E_X[E_{Y|X}[-log_2 p(Y|X)]] = - \sum_{m=1}^M p(x_m) \sum_{k=1}^K p(y_k | X=x_m) \log_2 p(y_k| X = x_m) \\
G(Y, X) = - \sum_{k=1}^K \sum_{m=1}^M p(y_k, x_m) \log_2 \frac{p(y_k) p(x_m)}{p(y_k, x_m)}
$$
通过公式推导，可证得：$G(Y,X) = H(Y) - H(Y|X)$

由于$H(Y,X) = \mathbb{E}_{(Y,X)\sim p(y,x)}[-\log_2p(Y,X)]$，可推导：
$$
\begin{aligned}
H(Y,X) 
&= \mathbb{E}_{(Y,X)\sim p(y,x)}[-\log_2p(Y,X)] \\
&= - \sum_{k=1}^K \sum_{m=1}^M p(y_k, x_m) \log_2 p(y_k, x_m) \\
&= - \sum_{k=1}^K \sum_{m=1}^M p(y_k, x_m) \log_2 \frac{p(x_m) p(y_k, x_m)}{p(x_m)} \\
&= - \sum_{k=1}^K \sum_{m=1}^M p(y_k, x_m) \left[ \log_2 p(x_m) + \log_2 \frac{p(y_k, x_m)}{p(x_m)} \right] \\
&= - \sum_{k=1}^K \sum_{m=1}^M p(y_k, x_m) \log_2 p(x_m) - \sum_{k=1}^K \sum_{m=1}^M p(y_k, x_m) \log_2 \frac{p(y_k, x_m)}{p(x_m)} \\
&= - \sum_{k=1}^K \sum_{m=1}^M p(y_k, x_m) \log_2 p(x_m) - \sum_{k=1}^K \sum_{m=1}^M p(x_m) \frac{p(y_k, x_m)}{p(x_m)} \log_2 \frac{p(y_k, x_m)}{p(x_m)} \\
&= H(X) + H(Y|X)
\end{aligned}
$$

$\therefore H(Y,X) = H(X) + H(Y|X)$ 

$
\begin{aligned}
\therefore G(Y,X) 
&= H(Y) - H(Y|X) \\
&= H(Y) - (H(Y, X) - H(X)) \\
&= H(X) + H(Y) - H(Y, X)
\end{aligned}$

$
\begin{aligned}
\because H(Y|X) = H(Y,X) - H(X) \\
H(X|Y) = H(X) - G(Y, X)
\end{aligned}
$

$\begin{aligned} 
\therefore & H(Y,X) -H(X|Y) - H(Y|X) \\
&= H(Y,X) - (H(X) - G(Y, X)) - (H(Y,X) - H(X)) \\
&= H(Y,X) - H(X) + G(Y,X) - H(Y,X) + H(X) \\
&= G(Y,X)
\end{aligned}
$

**第二问：**

1. $AB$区域代表$X$的不确定性，则$H(X)$表示$A \cup B$
2. $BC$区域代表$Y$的不确定性，则$H(Y)$表示$B \cup C$
3. $H(X|Y)$表示$A$区域
4. $H(Y|X)$表示$C$区域
5. $H(Y,X)$表示$A \cup B \cup C$区域
6. $G(Y,X)=H(Y) - H(Y|X)$表示$B$区域

### 2.2 练习2

假设当前我们需要处理一个分类问题，请问对输入特征进行归一化会对树模型的类别输出产生影响吗？请解释原因。

**解答：**

不会，因为归一化是一个线性变换，不会影响样本的输入特征和标签类别的分布。

### 2.3 练习3

如果将系数替换为$1- \gamma^2$，请问对缺失值是加强了还是削弱了惩罚？

**解答：**

在C4.5算法中，修正的信息增益是$$
\tilde{G}(Y,X) = (1 - \gamma)G(\tilde{Y}, \tilde{X})
$$
由于在相同的$\gamma$($\gamma<1$)下，则$1 - \gamma < 1 - \gamma^2$，信息增益增大了，对缺失值削弱了惩罚。

### 2.4 练习4

如果将树的生长策略从深度优先生长改为广度优先生长，假设其他参数保持不变的情况下，两个模型对应的结果输出可能不同吗？

**解答：**

两个模型对应的结果输出是一样的，树的生长对于树特征的分裂没有影响。

### 2.5 练习5

在一般的机器学习问题中，我们总是通过一组参数来定义模型的损失函数，并且在训练集上以最小化该损失函数为目标进行优化。请问对于决策树而言，模型优化的目标是什么？

**解答：**

对于决策树而言，模型优化的目标是信息增益最大化。

### 2.6 练习6

对信息熵中$\log$函数处进行一阶泰勒展开可以近似为基尼系数，那么如果在$p=1$处进行二阶泰勒展开我们可以获得什么近似指标？请写出对应指标的信息增益公式。

**解答：**

在$p=1$处进行二阶泰勒展开
$$
\begin{aligned}
H(Y) 
&= E_Y I(p) \\
&= E_Y[-\log_2 p(Y)] \\
&\approx E_Y[1 - p(Y) + \frac{1}{2}(1 - p(Y))^2] \\
&= \sum_{k=1}^K p(y_k)[1-p(y_k) + \frac{1}{2}(1 - p(y_k))^2]
\end{aligned}
$$

$$
\begin{aligned}
H(Y|X) 
&= E_X[E_{Y|X}(-\log_2 p(Y|X))] \\
&\approx E_X[E_{Y|X}[1 - p(Y|X) + \frac{1}{2}(1 - p(Y|X))^2]] \\
&= \sum_{m=1}^M p(x_m) \sum_{k=1}^K[p(y_k|X=x_m)(1 - p(y_k|X=x_m) + \frac{1}{2}(1 - p(y_k|X=x_m))^2) ]
\end{aligned}
$$

信息增益为$G(Y,X) = H(Y) - H(Y|X)$

### 2.7 练习7

除了信息熵和基尼系数之外，我们还可以使用节点的$1 - \max_k p(Y=y_k)$和第$m$个子节点的$1 - \max_k p(Y=y_k | X=x_m)$来作为衡量纯度的指标。请解释其合理性并给出相应的信息增益公式。

**解答：**

合理的原因：该纯度指标在类别特征单一分布时取最小值为0，在均匀分布时取最大值，并且该指标是单调函数。

先求出信息熵
$$
H(Y) = 1 - \max_k p(Y=y_k) \\
H(Y|X) = \sum_{m=1}^M p(x_m)[1 - \max_k p(Y=y_k | X=x_m)]
$$

信息增益为$G(Y,X) = H(Y) - H(Y|X)$

### 2.8 练习8

为什么对没有重复特征值的数据，决策树能够做到损失为0？

**解答：**

如果没有重复特征值，决策树的节点将会对每个叶节点进行划分，并且只有1个样本。

### 2.9 练习9

如何理解min_samples_leaf参数能够控制回归树输出值的平滑程度？

**解答：**

参数min_samples_leaf表示叶节点最小样本数，如果样本数越小，样本均值的估计的均方差越大，平滑程度就越低。

## 3 知识回顾

### 3.1 习题1

ID3树算法、C4.5树算法和CART算法之间有何异同？

**解答：**

相同点：
1. 都需要进行特征分裂
2. 都是选取最大的信息增益或信息增益比进行特征分裂
3. 在处理节点数值特征时，C4.5和CART都可以使用最佳分割法和随机分割法

不相同点：
1. ID3算法和C4.5算法只适用于分类问题，CART算法适用分类和回归问题
2. ID3算法通过选择最大信息增益对特征分裂，C4.5算法适用信息增益比，CART算法使用基于基尼系数的信息增益
3. C4.5算法可以处理数值特征、含缺失值的特征，样本缺失值越大，对信息增益的惩罚就越大

### 3.2 习题2

什么是信息增益？它衡量了什么指标？它有什么缺陷？

**解答：**

1. 信息增益：通过节点分裂之后带来了多少不确定性的降低或纯度的提高。
2. 衡量的指标：在得到了随机变量$X$的取值信息时，随机变量$Y$不确定性的平均减少量。
3. 缺陷：用信息增益来选择的决策树对类别较多的特征具有天然的倾向性，在类别占比均匀的情况下，类别数越多则熵越高，从而熵高的变量在信息增益上需要使用惩罚来抑制，在C4.5算法中，使用信息增益比代替信息增益

### 3.3 习题3

sklearn决策树中的random_state参数控制了哪些步骤的随机性？

**解答：**

1. 当max_features < n_features时，算法在每次特征分裂时随机选择max_features，从中找到最佳的特征。
2. 在splitter参数设置为best时，random_state将会控制该随机性。

### 3.4 习题4

决策树如何处理连续变量和缺失变量？

**解答：**

1. 连续变量的处理：通过使用随机分割法或最佳分割法
2. 缺失变量的处理：通过使用修正的信息增益：$$
\tilde{G}(Y,X) = (1 - \gamma)G(\tilde{Y}, \tilde{X})
$$其中，节点$N$的样本缺失值比例为$\gamma$，非缺失值对应的类别标签和特征分别为$\tilde{Y}$和$\tilde{X}$，当数据完全缺失时，$\gamma=1$，信息增益为0；当数据没有缺失值时，$\gamma=0$，信息增益与原来的值保持一致。

### 3.5 习题5

基尼系数是什么？为什么要在CART中引入它？

**解答：**

1. 基尼系数：$$\text{Gini}(Y)=1 - \sum_{k=1}^K \tilde{p}^2(y_k)$$
2. 条件基尼系数：$$\text{Gini}(Y|X) = \sum_{m=1}^M \tilde{p}(x_m) [1 - \sum_{k=1}^K \tilde{p}^2(y_k|x_m)]$$
3. 在CART引入的原因：在计算信息熵的时候，对数函数log函数计算代价较大，使用在$p=1$的一阶泰勒展开，进行熵的线性近似，降低计算复杂度。

### 3.6 习题6

什么是树的预剪枝和后剪枝？具体分别是如何操作的？

**解答：**

1. 预剪枝：
  - 概念：树在判断节点是否分裂的时候就预先通过一些规则来阻止其分裂。
  - 具体操作：通过6个控制参数，max_depth、min_samples_split、min_samples_leaf、min_weight_fraction_leaf、max_leaf_nodes、min_impurity_decrease
2. 后剪枝：
  - 概念：在树的节点已经全部生长完成后，同故宫一些规则来摘除一些子树。
  - 具体操作：通过ccp_alpha参数控制，计算剪枝度量，如果决策树某一个节点为根的剪枝度量小于子树的剪枝度量，保留子树，否则剪枝掉这棵子树中除根节点以外的所有节点。

## 4 总结

&emsp;&emsp;本次任务，主要介绍了决策树的相关知识，通过信息论的基础知识介绍，引入信息熵的概念，再根据信息增益，可使用两种算法（ID3、C4.5）处理节点分裂。使用CART树处理分类/回归问题，并引入基于基尼系数的信息增益处理对数函数计算代价问题；通过剪枝方法（预剪枝、后剪枝），解决树的无限生长问题，以防过拟合。